# Chapter 12: Route Handlers & APIs

While Server Actions are excellent for form submissions and mutations, Route Handlers provide a powerful way to create RESTful APIs in your Next.js application. They allow you to build backend endpoints that can be consumed by external clients, mobile applications, or third-party services.

By the end of this chapter, you'll master creating RESTful APIs, handling various HTTP methods, managing request/response cycles, and implementing secure, performant API endpoints.

## 12.1 Creating API Routes

In the Next.js App Router, API routes are created using Route Handlers. Unlike the Pages Router's `pages/api` directory, Route Handlers use the `route.ts` (or `route.js`) file convention within the `app` directory.

### Basic Route Handler Structure

Create API endpoints by defining a `route.ts` file in your app directory:

```typescript
// app/api/hello/route.ts
import { NextRequest, NextResponse } from 'next/server';

export async function GET(request: NextRequest) {
  return NextResponse.json({ 
    message: 'Hello from Next.js!',
    timestamp: new Date().toISOString(),
  });
}

export async function POST(request: NextRequest) {
  const body = await request.json();
  
  return NextResponse.json({ 
    message: 'Data received',
    data: body,
  }, { status: 201 });
}
```

**File Structure for APIs:**

```text
app/
├── api/
│   ├── hello/
│   │   └── route.ts          → /api/hello
│   ├── users/
│   │   └── route.ts          → /api/users (GET all, POST new)
│   └── users/
│       └── [id]/
│           └── route.ts      → /api/users/123 (GET, PUT, DELETE specific user)
```

### HTTP Methods Support

Route Handlers support all standard HTTP methods:

```typescript
// app/api/products/route.ts
import { NextRequest, NextResponse } from 'next/server';

// GET /api/products - List all products
export async function GET(request: NextRequest) {
  const { searchParams } = new URL(request.url);
  const category = searchParams.get('category');
  const limit = parseInt(searchParams.get('limit') || '10');
  
  try {
    const products = await db.product.findMany({
      where: category ? { category } : undefined,
      take: limit,
    });
    
    return NextResponse.json({ products });
  } catch (error) {
    return NextResponse.json(
      { error: 'Failed to fetch products' },
      { status: 500 }
    );
  }
}

// POST /api/products - Create new product
export async function POST(request: NextRequest) {
  try {
    const body = await request.json();
    
    // Validation
    if (!body.name || !body.price) {
      return NextResponse.json(
        { error: 'Name and price are required' },
        { status: 400 }
      );
    }
    
    const product = await db.product.create({
      data: {
        name: body.name,
        description: body.description,
        price: body.price,
        category: body.category,
      },
    });
    
    return NextResponse.json({ product }, { status: 201 });
  } catch (error) {
    return NextResponse.json(
      { error: 'Failed to create product' },
      { status: 500 }
    );
  }
}

// PUT /api/products - Bulk update (optional)
export async function PUT(request: NextRequest) {
  // Handle bulk updates
  return NextResponse.json({ message: 'Bulk update not implemented' }, { status: 501 });
}

// PATCH /api/products - Partial bulk update (optional)
export async function PATCH(request: NextRequest) {
  return NextResponse.json({ message: 'Bulk patch not implemented' }, { status: 501 });
}

// DELETE /api/products - Bulk delete (use with caution)
export async function DELETE(request: NextRequest) {
  return NextResponse.json({ message: 'Bulk delete not allowed' }, { status: 403 });
}
```

## 12.2 RESTful API Design

Designing clean, RESTful APIs follows specific conventions for resource management.

### Resource-Based Routing

Structure your API around resources (nouns), not actions (verbs):

```typescript
// Good RESTful design
// app/api/users/route.ts          → GET /api/users (list), POST /api/users (create)
// app/api/users/[id]/route.ts     → GET /api/users/123 (get), PUT /api/users/123 (update), DELETE /api/users/123 (delete)
// app/api/users/[id]/posts/route.ts → GET /api/users/123/posts (nested resource)

// Avoid verb-based routes like:
// /api/getUsers, /api/createUser, /api/deleteUser
```

### Complete CRUD Operations

Implement full CRUD for a resource:

```typescript
// app/api/articles/route.ts
import { NextRequest, NextResponse } from 'next/server';
import { z } from 'zod';

const articleSchema = z.object({
  title: z.string().min(1),
  content: z.string().min(1),
  authorId: z.string(),
  published: z.boolean().default(false),
  tags: z.array(z.string()).optional(),
});

// GET /api/articles
export async function GET(request: NextRequest) {
  try {
    const { searchParams } = new URL(request.url);
    
    // Pagination
    const page = parseInt(searchParams.get('page') || '1');
    const limit = parseInt(searchParams.get('limit') || '10');
    const skip = (page - 1) * limit;
    
    // Filtering
    const authorId = searchParams.get('authorId');
    const published = searchParams.get('published');
    const tag = searchParams.get('tag');
    
    // Sorting
    const sortBy = searchParams.get('sortBy') || 'createdAt';
    const order = searchParams.get('order') || 'desc';
    
    const where: any = {};
    if (authorId) where.authorId = authorId;
    if (published !== null) where.published = published === 'true';
    if (tag) where.tags = { has: tag };
    
    const [articles, total] = await Promise.all([
      db.article.findMany({
        where,
        skip,
        take: limit,
        orderBy: { [sortBy]: order },
        include: { author: { select: { id: true, name: true } } },
      }),
      db.article.count({ where }),
    ]);
    
    return NextResponse.json({
      articles,
      pagination: {
        page,
        limit,
        total,
        totalPages: Math.ceil(total / limit),
      },
    });
  } catch (error) {
    return NextResponse.json(
      { error: 'Failed to fetch articles' },
      { status: 500 }
    );
  }
}

// POST /api/articles
export async function POST(request: NextRequest) {
  try {
    const body = await request.json();
    const validated = articleSchema.parse(body);
    
    const article = await db.article.create({
      data: validated,
      include: { author: { select: { id: true, name: true } } },
    });
    
    return NextResponse.json({ article }, { status: 201 });
  } catch (error) {
    if (error instanceof z.ZodError) {
      return NextResponse.json(
        { error: 'Validation failed', details: error.errors },
        { status: 400 }
      );
    }
    
    return NextResponse.json(
      { error: 'Failed to create article' },
      { status: 500 }
    );
  }
}
```

```typescript
// app/api/articles/[id]/route.ts
import { NextRequest, NextResponse } from 'next/server';

interface RouteParams {
  params: { id: string };
}

// GET /api/articles/[id]
export async function GET(request: NextRequest, { params }: RouteParams) {
  try {
    const article = await db.article.findUnique({
      where: { id: params.id },
      include: {
        author: { select: { id: true, name: true, email: true } },
        comments: {
          orderBy: { createdAt: 'desc' },
          take: 10,
        },
      },
    });
    
    if (!article) {
      return NextResponse.json(
        { error: 'Article not found' },
        { status: 404 }
      );
    }
    
    return NextResponse.json({ article });
  } catch (error) {
    return NextResponse.json(
      { error: 'Failed to fetch article' },
      { status: 500 }
    );
  }
}

// PUT /api/articles/[id] - Full update
export async function PUT(request: NextRequest, { params }: RouteParams) {
  try {
    const body = await request.json();
    const validated = articleSchema.parse(body);
    
    const article = await db.article.update({
      where: { id: params.id },
      data: validated,
    });
    
    return NextResponse.json({ article });
  } catch (error) {
    if (error.code === 'P2025') { // Prisma not found error
      return NextResponse.json(
        { error: 'Article not found' },
        { status: 404 }
      );
    }
    
    return NextResponse.json(
      { error: 'Failed to update article' },
      { status: 500 }
    );
  }
}

// PATCH /api/articles/[id] - Partial update
export async function PATCH(request: NextRequest, { params }: RouteParams) {
  try {
    const body = await request.json();
    
    // Partial validation - only validate provided fields
    const validated = articleSchema.partial().parse(body);
    
    const article = await db.article.update({
      where: { id: params.id },
      data: validated,
    });
    
    return NextResponse.json({ article });
  } catch (error) {
    return NextResponse.json(
      { error: 'Failed to update article' },
      { status: 500 }
    );
  }
}

// DELETE /api/articles/[id]
export async function DELETE(request: NextRequest, { params }: RouteParams) {
  try {
    await db.article.delete({
      where: { id: params.id },
    });
    
    return NextResponse.json(
      { message: 'Article deleted successfully' },
      { status: 200 }
    );
  } catch (error) {
    if (error.code === 'P2025') {
      return NextResponse.json(
        { error: 'Article not found' },
        { status: 404 }
      );
    }
    
    return NextResponse.json(
      { error: 'Failed to delete article' },
      { status: 500 }
    );
  }
}
```

## 12.3 Handling HTTP Methods

Each HTTP method serves a specific purpose in RESTful APIs.

### GET Requests with Query Parameters

Handle complex filtering, sorting, and pagination:

```typescript
// app/api/search/route.ts
import { NextRequest, NextResponse } from 'next/server';

export async function GET(request: NextRequest) {
  const { searchParams } = new URL(request.url);
  
  // Search query
  const query = searchParams.get('q') || '';
  
  // Filters
  const filters = {
    category: searchParams.getAll('category'),
    priceMin: searchParams.get('price_min') ? parseFloat(searchParams.get('price_min')!) : undefined,
    priceMax: searchParams.get('price_max') ? parseFloat(searchParams.get('price_max')!) : undefined,
    inStock: searchParams.get('in_stock') === 'true',
    rating: searchParams.get('rating') ? parseInt(searchParams.get('rating')!) : undefined,
  };
  
  // Sorting
  const sort = {
    field: searchParams.get('sort') || 'relevance',
    order: searchParams.get('order') || 'desc',
  };
  
  // Pagination
  const page = parseInt(searchParams.get('page') || '1');
  const limit = Math.min(parseInt(searchParams.get('limit') || '20'), 100); // Max 100
  const offset = (page - 1) * limit;
  
  try {
    // Build where clause
    const where: any = {};
    
    if (query) {
      where.OR = [
        { name: { contains: query, mode: 'insensitive' } },
        { description: { contains: query, mode: 'insensitive' } },
      ];
    }
    
    if (filters.category.length > 0) {
      where.category = { in: filters.category };
    }
    
    if (filters.priceMin !== undefined || filters.priceMax !== undefined) {
      where.price = {};
      if (filters.priceMin !== undefined) where.price.gte = filters.priceMin;
      if (filters.priceMax !== undefined) where.price.lte = filters.priceMax;
    }
    
    if (filters.inStock) {
      where.stock = { gt: 0 };
    }
    
    if (filters.rating) {
      where.rating = { gte: filters.rating };
    }
    
    const [results, total] = await Promise.all([
      db.product.findMany({
        where,
        skip: offset,
        take: limit,
        orderBy: sort.field === 'relevance' 
          ? { createdAt: 'desc' } 
          : { [sort.field]: sort.order },
      }),
      db.product.count({ where }),
    ]);
    
    return NextResponse.json({
      results,
      meta: {
        query,
        filters,
        sort,
        pagination: {
          page,
          limit,
          total,
          totalPages: Math.ceil(total / limit),
          hasNextPage: page * limit < total,
          hasPrevPage: page > 1,
        },
      },
    });
  } catch (error) {
    return NextResponse.json(
      { error: 'Search failed' },
      { status: 500 }
    );
  }
}
```

### POST Requests with File Uploads

Handle multipart form data for file uploads:

```typescript
// app/api/upload/route.ts
import { NextRequest, NextResponse } from 'next/server';
import { writeFile } from 'fs/promises';
import { join } from 'path';
import { v4 as uuidv4 } from 'uuid';

export async function POST(request: NextRequest) {
  try {
    const formData = await request.formData();
    
    const file = formData.get('file') as File;
    const folder = formData.get('folder') as string || 'uploads';
    
    if (!file) {
      return NextResponse.json(
        { error: 'No file provided' },
        { status: 400 }
      );
    }
    
    // Validate file
    const allowedTypes = ['image/jpeg', 'image/png', 'image/webp', 'application/pdf'];
    if (!allowedTypes.includes(file.type)) {
      return NextResponse.json(
        { error: 'Invalid file type' },
        { status: 400 }
      );
    }
    
    const maxSize = 10 * 1024 * 1024; // 10MB
    if (file.size > maxSize) {
      return NextResponse.json(
        { error: 'File too large' },
        { status: 400 }
      );
    }
    
    // Generate unique filename
    const bytes = await file.arrayBuffer();
    const buffer = Buffer.from(bytes);
    const filename = `${uuidv4()}-${file.name}`;
    const path = join(process.cwd(), 'public', folder, filename);
    
    // Save file
    await writeFile(path, buffer);
    
    // Save metadata to database
    const fileRecord = await db.file.create({
      data: {
        filename: file.name,
        path: `/${folder}/${filename}`,
        size: file.size,
        mimetype: file.type,
        uploadedAt: new Date(),
      },
    });
    
    return NextResponse.json({
      success: true,
      file: {
        id: fileRecord.id,
        url: `/${folder}/${filename}`,
        name: file.name,
        size: file.size,
      },
    }, { status: 201 });
    
  } catch (error) {
    return NextResponse.json(
      { error: 'Upload failed' },
      { status: 500 }
    );
  }
}
```

### Handling Different Content Types

Support various request body formats:

```typescript
// app/api/webhooks/route.ts
import { NextRequest, NextResponse } from 'next/server';
import { headers } from 'next/headers';
import crypto from 'crypto';

export async function POST(request: NextRequest) {
  const headersList = headers();
  const contentType = headersList.get('content-type') || '';
  
  try {
    let body: any;
    
    // Handle different content types
    if (contentType.includes('application/json')) {
      body = await request.json();
    } else if (contentType.includes('application/x-www-form-urlencoded')) {
      const formData = await request.formData();
      body = Object.fromEntries(formData);
    } else if (contentType.includes('text/plain')) {
      body = await request.text();
    } else if (contentType.includes('application/xml') || contentType.includes('text/xml')) {
      body = await request.text();
      // Parse XML if needed
    } else {
      // Raw body for webhooks with signatures
      const rawBody = await request.text();
      body = rawBody;
      
      // Verify webhook signature (Stripe example)
      const signature = headersList.get('stripe-signature');
      if (signature) {
        const isValid = verifyStripeSignature(rawBody, signature);
        if (!isValid) {
          return NextResponse.json(
            { error: 'Invalid signature' },
            { status: 401 }
          );
        }
        body = JSON.parse(rawBody);
      }
    }
    
    // Process webhook
    await processWebhook(body);
    
    return NextResponse.json({ received: true });
    
  } catch (error) {
    return NextResponse.json(
      { error: 'Webhook processing failed' },
      { status: 500 }
    );
  }
}

function verifyStripeSignature(payload: string, signature: string): boolean {
  const secret = process.env.STRIPE_WEBHOOK_SECRET!;
  const expectedSignature = crypto
    .createHmac('sha256', secret)
    .update(payload)
    .digest('hex');
    
  return crypto.timingSafeEqual(
    Buffer.from(signature),
    Buffer.from(expectedSignature)
  );
}
```

## 12.4 Request and Response Objects

Understanding the NextRequest and NextResponse objects is essential for building robust APIs.

### Working with NextRequest

Access request details:

```typescript
// app/api/debug/route.ts
import { NextRequest, NextResponse } from 'next/server';
import { headers } from 'next/headers';

export async function GET(request: NextRequest) {
  // URL and query params
  const url = request.url;
  const { searchParams } = new URL(url);
  
  // Headers
  const headersList = headers();
  const userAgent = headersList.get('user-agent');
  const authorization = headersList.get('authorization');
  const contentType = headersList.get('content-type');
  
  // IP address (when behind proxy)
  const ip = request.ip || headersList.get('x-forwarded-for')?.split(',')[0]?.trim();
  
  // Geo information (Vercel specific)
  const geo = request.geo;
  
  // Cookies
  const cookieStore = request.cookies;
  const sessionId = cookieStore.get('session-id')?.value;
  
  // Next.js specific
  const geoData = request.geo;
  
  return NextResponse.json({
    url,
    query: Object.fromEntries(searchParams),
    headers: {
      userAgent,
      contentType,
      authorization: authorization ? 'Present' : 'Missing',
    },
    ip,
    geo: geoData,
    cookies: {
      sessionId: sessionId ? 'Present' : 'Missing',
    },
    nextUrl: {
      pathname: request.nextUrl.pathname,
      search: request.nextUrl.search,
    },
  });
}
```

### Custom NextResponse

Create sophisticated responses:

```typescript
// app/api/download/route.ts
import { NextRequest, NextResponse } from 'next/server';
import { readFile } from 'fs/promises';
import { join } from 'path';

export async function GET(request: NextRequest) {
  const { searchParams } = new URL(request.url);
  const filename = searchParams.get('file');
  
  if (!filename) {
    return NextResponse.json(
      { error: 'Filename required' },
      { status: 400 }
    );
  }
  
  try {
    const filePath = join(process.cwd(), 'files', filename);
    const fileBuffer = await readFile(filePath);
    
    // Determine content type
    const ext = filename.split('.').pop()?.toLowerCase();
    const contentType = {
      pdf: 'application/pdf',
      png: 'image/png',
      jpg: 'image/jpeg',
      json: 'application/json',
    }[ext] || 'application/octet-stream';
    
    // Create response with custom headers
    const response = new NextResponse(fileBuffer, {
      status: 200,
      headers: {
        'Content-Type': contentType,
        'Content-Disposition': `attachment; filename="${filename}"`,
        'Content-Length': fileBuffer.length.toString(),
        'Cache-Control': 'private, max-age=3600',
        'X-Content-Type-Options': 'nosniff',
      },
    });
    
    return response;
    
  } catch (error) {
    return NextResponse.json(
      { error: 'File not found' },
      { status: 404 }
    );
  }
}
```

### Response Streaming

Stream large responses:

```typescript
// app/api/stream/route.ts
import { NextRequest, NextResponse } from 'next/server';

export async function GET(request: NextRequest) {
  const encoder = new TextEncoder();
  
  const stream = new ReadableStream({
    async start(controller) {
      // Send data in chunks
      for (let i = 0; i < 10; i++) {
        const data = {
          chunk: i,
          timestamp: new Date().toISOString(),
          data: `Chunk ${i} of data`,
        };
        
        controller.enqueue(
          encoder.encode(`data: ${JSON.stringify(data)}\n\n`)
        );
        
        // Simulate delay
        await new Promise(resolve => setTimeout(resolve, 1000));
      }
      
      controller.close();
    },
  });
  
  return new NextResponse(stream, {
    headers: {
      'Content-Type': 'text/event-stream',
      'Cache-Control': 'no-cache',
      'Connection': 'keep-alive',
    },
  });
}
```

## 12.5 CORS and Security Headers

Secure your APIs with proper CORS configuration and security headers.

### CORS Configuration

```typescript
// middleware.ts
import { NextResponse } from 'next/server';
import type { NextRequest } from 'next/server';

const allowedOrigins = [
  'https://app.yoursite.com',
  'https://admin.yoursite.com',
  'http://localhost:3000',
];

export function middleware(request: NextRequest) {
  const origin = request.headers.get('origin') || '';
  const isAllowedOrigin = allowedOrigins.includes(origin);
  
  // Handle preflight requests
  if (request.method === 'OPTIONS') {
    const response = new NextResponse(null, { status: 204 });
    
    if (isAllowedOrigin) {
      response.headers.set('Access-Control-Allow-Origin', origin);
    }
    
    response.headers.set('Access-Control-Allow-Methods', 'GET, POST, PUT, PATCH, DELETE, OPTIONS');
    response.headers.set('Access-Control-Allow-Headers', 'Content-Type, Authorization');
    response.headers.set('Access-Control-Max-Age', '86400');
    
    return response;
  }
  
  // Handle actual requests
  const response = NextResponse.next();
  
  if (isAllowedOrigin) {
    response.headers.set('Access-Control-Allow-Origin', origin);
  }
  
  response.headers.set('Access-Control-Allow-Credentials', 'true');
  
  return response;
}

export const config = {
  matcher: '/api/:path*',
};
```

### Security Headers in Route Handlers

```typescript
// lib/security.ts
import { NextResponse } from 'next/server';

export function addSecurityHeaders(response: NextResponse): NextResponse {
  // Prevent XSS
  response.headers.set('X-XSS-Protection', '1; mode=block');
  
  // Prevent clickjacking
  response.headers.set('X-Frame-Options', 'DENY');
  
  // Prevent MIME type sniffing
  response.headers.set('X-Content-Type-Options', 'nosniff');
  
  // Enforce HTTPS
  response.headers.set('Strict-Transport-Security', 'max-age=31536000; includeSubDomains');
  
  // Content Security Policy
  response.headers.set(
    'Content-Security-Policy',
    "default-src 'self'; script-src 'self' 'unsafe-eval' 'unsafe-inline'; style-src 'self' 'unsafe-inline'"
  );
  
  // Referrer Policy
  response.headers.set('Referrer-Policy', 'strict-origin-when-cross-origin');
  
  // Permissions Policy
  response.headers.set(
    'Permissions-Policy',
    'camera=(), microphone=(), geolocation=(self)'
  );
  
  return response;
}
```

```typescript
// app/api/secure/route.ts
import { NextRequest, NextResponse } from 'next/server';
import { addSecurityHeaders } from '@/lib/security';

export async function GET(request: NextRequest) {
  const response = NextResponse.json({
    message: 'Secure data',
    timestamp: new Date().toISOString(),
  });
  
  return addSecurityHeaders(response);
}
```

## 12.6 Dynamic API Routes

Create dynamic endpoints for specific resources.

### Nested Dynamic Routes

```typescript
// app/api/users/[userId]/posts/[postId]/route.ts
import { NextRequest, NextResponse } from 'next/server';

interface RouteParams {
  params: {
    userId: string;
    postId: string;
  };
}

export async function GET(request: NextRequest, { params }: RouteParams) {
  try {
    // Verify user exists
    const user = await db.user.findUnique({
      where: { id: params.userId },
    });
    
    if (!user) {
      return NextResponse.json(
        { error: 'User not found' },
        { status: 404 }
      );
    }
    
    // Get specific post belonging to user
    const post = await db.post.findFirst({
      where: {
        id: params.postId,
        authorId: params.userId,
      },
      include: {
        author: {
          select: { id: true, name: true, email: true },
        },
        comments: {
          orderBy: { createdAt: 'desc' },
          take: 5,
        },
      },
    });
    
    if (!post) {
      return NextResponse.json(
        { error: 'Post not found' },
        { status: 404 }
      );
    }
    
    return NextResponse.json({ post });
  } catch (error) {
    return NextResponse.json(
      { error: 'Failed to fetch post' },
      { status: 500 }
    );
  }
}
```

### Catch-All Routes

Handle variable path segments:

```typescript
// app/api/files/[...path]/route.ts
import { NextRequest, NextResponse } from 'next/server';
import { readFile } from 'fs/promises';
import { join } from 'path';

export async function GET(request: NextRequest, { params }: { params: { path: string[] } }) {
  try {
    // Reconstruct file path from array
    const filePath = join(process.cwd(), 'storage', ...params.path);
    
    // Security: Ensure path doesn't escape storage directory
    const storageRoot = join(process.cwd(), 'storage');
    if (!filePath.startsWith(storageRoot)) {
      return NextResponse.json(
        { error: 'Invalid path' },
        { status: 403 }
      );
    }
    
    const file = await readFile(filePath);
    
    return new NextResponse(file, {
      headers: {
        'Content-Type': 'application/octet-stream',
        'Content-Disposition': `attachment; filename="${params.path.pop()}"`,
      },
    });
  } catch (error) {
    return NextResponse.json(
      { error: 'File not found' },
      { status: 404 }
    );
  }
}
```

## 12.7 API Route Best Practices

Follow these patterns for production-ready APIs.

### Input Validation

Always validate and sanitize inputs:

```typescript
// lib/validation.ts
import { z } from 'zod';

export const createUserSchema = z.object({
  email: z.string().email(),
  name: z.string().min(2).max(100),
  role: z.enum(['user', 'admin', 'editor']),
  age: z.number().min(18).optional(),
});

export function validateInput<T>(schema: z.ZodSchema<T>, data: unknown): T {
  return schema.parse(data);
}
```

### Consistent Error Responses

Standardize error format:

```typescript
// lib/api-response.ts
import { NextResponse } from 'next/server';

export class ApiError extends Error {
  constructor(
    public statusCode: number,
    message: string,
    public details?: any
  ) {
    super(message);
  }
}

export function successResponse(data: any, status: number = 200) {
  return NextResponse.json(
    {
      success: true,
      data,
      timestamp: new Date().toISOString(),
    },
    { status }
  );
}

export function errorResponse(error: ApiError | Error, statusCode: number = 500) {
  const isApiError = error instanceof ApiError;
  
  return NextResponse.json(
    {
      success: false,
      error: {
        message: error.message,
        code: isApiError ? error.statusCode : statusCode,
        details: isApiError ? error.details : undefined,
      },
      timestamp: new Date().toISOString(),
    },
    { status: isApiError ? error.statusCode : statusCode }
  );
}
```

### Rate Limiting

Protect your API from abuse:

```typescript
// lib/rate-limit.ts
import { LRUCache } from 'lru-cache';

type RateLimitOptions = {
  uniqueTokenPerInterval?: number;
  interval?: number;
};

export function rateLimit(options?: RateLimitOptions) {
  const tokenCache = new LRUCache({
    max: options?.uniqueTokenPerInterval || 500,
    ttl: options?.interval || 60000, // 1 minute
  });

  return {
    check: (token: string, limit: number) => {
      const tokenCount = (tokenCache.get(token) as number[]) || [0];
      if (tokenCount[0] === 0) {
        tokenCache.set(token, tokenCount);
      }
      tokenCount[0] += 1;

      const currentUsage = tokenCount[0];
      const isRateLimited = currentUsage > limit;
      
      return {
        isRateLimited,
        limit,
        remaining: isRateLimited ? 0 : limit - currentUsage,
        reset: Date.now() + (options?.interval || 60000),
      };
    },
  };
}
```

```typescript
// Usage in API route
import { rateLimit } from '@/lib/rate-limit';
import { errorResponse } from '@/lib/api-response';

const limiter = rateLimit({
  interval: 60 * 1000, // 1 minute
  uniqueTokenPerInterval: 500,
});

export async function POST(request: NextRequest) {
  try {
    const ip = request.ip || 'anonymous';
    const { isRateLimited, remaining } = limiter.check(ip, 10); // 10 requests per minute
    
    if (isRateLimited) {
      return errorResponse(
        new ApiError(429, 'Too many requests'),
        429
      );
    }
    
    // Process request...
    
    const response = NextResponse.json({ success: true });
    response.headers.set('X-RateLimit-Limit', '10');
    response.headers.set('X-RateLimit-Remaining', remaining.toString());
    
    return response;
  } catch (error) {
    return errorResponse(error as Error);
  }
}
```

### Authentication Middleware

Protect routes with authentication:

```typescript
// app/api/protected/route.ts
import { NextRequest, NextResponse } from 'next/server';
import { verifyToken } from '@/lib/auth';
import { errorResponse } from '@/lib/api-response';

export async function GET(request: NextRequest) {
  try {
    // Extract token from header
    const authHeader = request.headers.get('authorization');
    if (!authHeader?.startsWith('Bearer ')) {
      return errorResponse(
        new ApiError(401, 'Missing or invalid authorization header'),
        401
      );
    }
    
    const token = authHeader.split(' ')[1];
    const payload = await verifyToken(token);
    
    if (!payload) {
      return errorResponse(
        new ApiError(401, 'Invalid or expired token'),
        401
      );
    }
    
    // Token is valid, proceed with request
    const userData = await db.user.findUnique({
      where: { id: payload.userId },
      select: { id: true, email: true, role: true },
    });
    
    return NextResponse.json({
      message: 'Protected data',
      user: userData,
    });
    
  } catch (error) {
    return errorResponse(error as Error);
  }
}
```

## Key Takeaways from Chapter 12

1. **Creating API Routes**: Route Handlers in Next.js App Router use the `route.ts` file convention within the `app/api` directory, supporting all HTTP methods (GET, POST, PUT, PATCH, DELETE) with automatic request/response handling.

2. **RESTful API Design**: Structure APIs around resources using nouns in URLs, implement full CRUD operations, support filtering/sorting/pagination via query parameters, and return consistent response formats with proper HTTP status codes.

3. **Handling HTTP Methods**: Each HTTP method serves a specific purpose—GET for reading, POST for creating, PUT for full updates, PATCH for partial updates, and DELETE for removal. Handle various content types including JSON, form data, and file uploads.

4. **Request and Response Objects**: Use `NextRequest` for accessing URL parameters, headers, cookies, and body content. Use `NextResponse` for sending JSON responses, custom headers, streaming data, and file downloads.

5. **CORS and Security Headers**: Configure CORS in middleware.ts to handle cross-origin requests, implement security headers (XSS protection, frame options, content type options), and use Content Security Policy to prevent injection attacks.

6. **Dynamic API Routes**: Create dynamic endpoints using `[param]` syntax for single segments and `[...path]` for catch-all routes. Validate and sanitize dynamic parameters to prevent path traversal attacks.

7. **API Route Best Practices**: Always validate input using Zod schemas, standardize error responses, implement rate limiting to prevent abuse, authenticate requests using tokens or sessions, and organize code with proper separation of concerns.

## Coming Up Next

**Chapter 13: Rendering Strategies**

Now that you've mastered API routes, it's time to understand the various rendering strategies available in Next.js. In Chapter 13, we'll explore Static Site Generation (SSG), Server-Side Rendering (SSR), Incremental Static Regeneration (ISR), Client-Side Rendering (CSR), and how to choose the right strategy for each page. You'll learn to build applications that combine these strategies for optimal performance and user experience.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='11. server_actions.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='13. rendering_strategies.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
